# Clase: Toma de decisiones con IA

Ejemplo simple de **árbol de decisión** para mostrar en clases.

Escenario ficticio: *riesgo de abandono escolar* (similar al ejemplo de clasificación, pero ahora usando un árbol de decisión y viéndolo de forma visual).

## 1. Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix


## 2. Crear dataset ficticio

Usaremos el mismo escenario:
- `asistencia_pct`
- `promedio_nota`
- `reprobaciones`
- `apoyo_familiar`

Etiqueta: `riesgo_alto` (1 = alto riesgo de deserción, 0 = bajo).

In [ ]:
np.random.seed(42)
n = 400

asistencia = np.clip(np.random.normal(85, 10, n), 50, 100)
promedio = np.clip(np.random.normal(5, 1, n), 2, 7)
reprobaciones = np.random.poisson(0.5, n)
apoyo_familiar = np.random.binomial(1, 0.6, n)

puntaje_riesgo = (
    (asistencia < 80).astype(int) +
    (promedio < 4.5).astype(int) +
    (reprobaciones >= 1).astype(int) +
    (apoyo_familiar == 0).astype(int)
)

riesgo_alto = (puntaje_riesgo >= 2).astype(int)

df = pd.DataFrame({
    "asistencia_pct": asistencia,
    "promedio_nota": promedio,
    "reprobaciones": reprobaciones,
    "apoyo_familiar": apoyo_familiar,
    "riesgo_alto": riesgo_alto
})

df.head()


## 3. Separar datos de entrada y salida

In [ ]:
X = df[["asistencia_pct", "promedio_nota", "reprobaciones", "apoyo_familiar"]]
y = df["riesgo_alto"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_train.shape, X_test.shape


## 4. Entrenar un árbol de decisión

In [ ]:
arbol = DecisionTreeClassifier(
    max_depth=3,      # para que sea fácil de visualizar
    random_state=42
)

arbol.fit(X_train, y_train)
print("Árbol entrenado 👍")


## 5. Evaluar el árbol

In [ ]:
y_pred = arbol.predict(X_test)

print("Informe de clasificación:\n")
print(classification_report(y_test, y_pred))

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))


## 6. Visualizar el árbol

Este gráfico es el que puedes mostrar en pantalla para explicar la lógica paso a paso.

In [ ]:
plt.figure(figsize=(12, 6))
plot_tree(
    arbol,
    feature_names=X.columns,
    class_names=["bajo_riesgo", "alto_riesgo"],
    filled=True,
    rounded=True
)
plt.title("Árbol de decisión: riesgo de abandono escolar")
plt.show()


## 7. Probar con casos individuales

Para mostrar cómo el árbol decide con ejemplos concretos.

In [ ]:
caso_bajo_riesgo = pd.DataFrame({
    "asistencia_pct": [92],
    "promedio_nota": [6.0],
    "reprobaciones": [0],
    "apoyo_familiar": [1]
})

caso_alto_riesgo = pd.DataFrame({
    "asistencia_pct": [68],
    "promedio_nota": [3.5],
    "reprobaciones": [2],
    "apoyo_familiar": [0]
})

print("Predicción caso 1 (0=bajo,1=alto):", arbol.predict(caso_bajo_riesgo)[0])
print("Predicción caso 2 (0=bajo,1=alto):", arbol.predict(caso_alto_riesgo)[0])


## 8. Resumen para la clase

- El árbol aprende **reglas de decisión** a partir de los datos.
- Cada nodo del árbol es una pregunta (ej: `promedio_nota < 4.7?`).
- Cada rama es una respuesta (sí/no) que lleva a otra pregunta o a una hoja final.
- Es un modelo muy útil cuando se necesita **explicar** por qué se tomó una decisión (trazabilidad).